### <font color=black> GET:: <font color=blue> **path** </font> of current ipynb</font>

In [ ]:
# import os
# from ipykernel import get_connection_file

# # Get the name and location of the current Jupyter notebook
# notebook_path = os.path.dirname(get_connection_file())

# # Print the name and location of the current Jupyter notebook
# print("Notebook name:", os.path.basename(notebook_path))
# print("Notebook location:", notebook_path)

### <font color=black> IMPORT:: <font color=blue> **py modules** </font> </font>

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from mpl_toolkits.basemap import Basemap
import glob
import os
import shutil
import re
import cartopy
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
%matplotlib inline
#
PD=os.getcwd(); PD


'/scratch/pk695/FACTS/002_fork/facts/JupNbk/000_pk-JupNb_TESTspace/2023-01_NZ_INSAR'

### <font color=black> **====================================================================** </font>
### <font size=5 color="black"><b>FACTS EXP::</b></font> <font size=5 color="blue"><b>NZrerun1</b></font> <font size=2> [ NZ module with Ian's updated Insar dataset ]</font>


In [ ]:
#path='/scratch/pk695/FACTS/002_fork/facts/experiments-pkjr002/NZrerun/'
#fnames = glob.glob(path+'output_set_*')
#fnames
# wf_id = [fname.split('_')[0] for fname in fnames]
# wf_id

##### <font size=5 color="black"> FETCH:: </font> <font size=5 color="blue"><b> EXP files</b></font> 

In [2]:
path='/scratch/pk695/FACTS/002_fork/facts/experiments-pkjr002/NZrerun/000_TEST_NZInsarGPS/'
expnames=[d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d)) and re.match(r'^NZrerun.ssp\d{3}$', d)]
expnames=sorted(expnames); print(expnames)

['NZrerun.ssp119', 'NZrerun.ssp126', 'NZrerun.ssp245', 'NZrerun.ssp370', 'NZrerun.ssp585']


### <font color=black> GET output sub-folder list <font color=blue> (e.g., output_0.txt, output_1.txt ...) </font>  </font>

In [3]:
exp='NZrerun.ssp585'
exppath=path+exp
#
if (exp=='NZrerun.ssp585') or (exp=='NZrerun.ssp370'):
    OPloopdir = [d for d in os.listdir(exppath) if os.path.isdir(os.path.join(exppath, d)) and re.match(r'^output_\d+\.txt$', d)]
    OPloopdir.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))
else:
    OPloopdir = [d for d in os.listdir(exppath) if os.path.isdir(os.path.join(exppath, d)) and re.match(r'^output_set_\d', d)]
    OPloopdir.sort(key=lambda x: int(x.split('_')[2]))
print(OPloopdir)

['output_0.txt', 'output_1.txt', 'output_2.txt', 'output_3.txt', 'output_4.txt', 'output_5.txt', 'output_6.txt', 'output_7.txt', 'output_8.txt', 'output_9.txt', 'output_10.txt', 'output_11.txt', 'output_12.txt', 'output_13.txt', 'output_14.txt']


### <font color=black> GET <font color=blue> *.nc file NAMES </font> to merge </font>

In [4]:
# Get all the *.nc file names (can use OPloopdir[n])
nclist=os.listdir(path+exp+'/'+OPloopdir[0]+'/');  nclist=sorted(nclist)
nclist_local=[]
for ncname in nclist:
    if 'local' in ncname:
        nclist_local.append(ncname) 
nclist_local

['NZrerun.ssp585.GrIS1f.FittedISMIP.GrIS_GIS_localsl.nc',
 'NZrerun.ssp585.NZInsarGPS.NZInsarGPS.verticallandmotion_localsl.nc',
 'NZrerun.ssp585.ar5AIS.ipccar5.icesheets_EAIS_localsl.nc',
 'NZrerun.ssp585.ar5AIS.ipccar5.icesheets_WAIS_localsl.nc',
 'NZrerun.ssp585.ar5glaciers.ipccar5.glaciers_localsl.nc',
 'NZrerun.ssp585.bamber19.bamber19.icesheets_AIS_localsl.nc',
 'NZrerun.ssp585.bamber19.bamber19.icesheets_EAIS_localsl.nc',
 'NZrerun.ssp585.bamber19.bamber19.icesheets_GIS_localsl.nc',
 'NZrerun.ssp585.bamber19.bamber19.icesheets_WAIS_localsl.nc',
 'NZrerun.ssp585.deconto21.deconto21.AIS_AIS_localsl.nc',
 'NZrerun.ssp585.deconto21.deconto21.AIS_EAIS_localsl.nc',
 'NZrerun.ssp585.deconto21.deconto21.AIS_WAIS_localsl.nc',
 'NZrerun.ssp585.emuAIS.emulandice.AIS_EAIS_localsl.nc',
 'NZrerun.ssp585.emuAIS.emulandice.AIS_WAIS_localsl.nc',
 'NZrerun.ssp585.emuAIS.emulandice.AIS_localsl.nc',
 'NZrerun.ssp585.emuGrIS.emulandice.GrIS_localsl.nc',
 'NZrerun.ssp585.emuglaciers.emulandice.glacie

### <font color=black> **====================================================================** </font>
### <font color=black> LoopOver <font color=blue> **OPdir** </font> to merge each *.nc file</font>

In [13]:
# Fetch EXP files
#path='/scratch/pk695/FACTS/002_fork/facts/experiments-pkjr002/NZrerun/000_TEST_NZInsarGPS/'
#expnames=[d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d)) and re.match(r'^NZrerun.ssp\d{3}$', d)]
path='/scratch/pk695/FACTS/002_fork/facts/experiments-pkjr002/NZrerun/'
expnames=[d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d)) and re.match(r'^NZrerun1.ssp\d{3}$', d)]
expnames=sorted(expnames); print(expnames)

# Loop over each EXP
for x_exp,exp in enumerate(expnames):
    #
    # List the OP folder files
    OPloopdir=[]
    OPloopdir = [d for d in os.listdir(path+exp) if os.path.isdir(os.path.join(path+exp, d)) and re.match(r'^output_set_\d+', d)]
    OPloopdir.sort(key=lambda x: int(x.split('_')[2]))
    #print(OPloopdir)
    #
    # Make OP folders to save the new merged .nc files
    stackfolderLocal=path + exp + '/' + 'output_local_' + exp.split('.')[1]
    stackfolderGlobal=path + exp + '/' + 'output_global_' + exp.split('.')[1]
    # If the directory already exists, remove it and its contents
    if os.path.exists(stackfolderLocal):   shutil.rmtree(stackfolderLocal)
    os.makedirs(stackfolderLocal)
    if os.path.exists(stackfolderGlobal):  shutil.rmtree(stackfolderGlobal); 
    os.makedirs(stackfolderGlobal)
    #
    # Get all the *.nc file names within each output directory (can use OPloopdir[n] as default)
    nclist=os.listdir(path+exp+'/'+OPloopdir[0]+'/');  nclist=sorted(nclist)
    nclist_local=[]; nclist_global=[]
    for ncname in nclist:
        if 'local' in ncname:
            nclist_local.append(ncname) 
        elif 'global' in ncname:
            nclist_global.append(ncname)
    #nclist_local; #nclist_global
    
    # Stack the local files from each output set within single EXP (i.e. ssp)
    stackNC(nclist_local,stackfolderLocal,OPloopdir)
    
    # Global doesent loop over OPloopdir
    src_dir = path+exp+'/'+OPloopdir[0]+'/'
    # Destination directory
    dst_dir = stackfolderGlobal+'/'
    # Loop through the list of files and copy them to the destination directory
    for file in nclist_global:
        src_path = src_dir + file
        dst_path = dst_dir + file
        shutil.copy(src_path, dst_path)
    
    

['NZrerun1.ssp119', 'NZrerun1.ssp126', 'NZrerun1.ssp245', 'NZrerun1.ssp370', 'NZrerun1.ssp585']


In [12]:
dst_path = dst_dir + file
dst_path

'/scratch/pk695/FACTS/002_fork/facts/experiments-pkjr002/NZrerun/NZrerun1.ssp119/output_global_ssp119/NZrerun1.ssp119c.GrIS1f.FittedISMIP.GrIS_GIS_globalsl.nc'

In [6]:
def stackNC(filein,stackfolder,OPLOOPDIR):
    for x_ncfile,ncfile in enumerate(filein):
        file1=ncfile
        #
        #loop through output folders
        ds1_list=[]  
        for x_op,op in enumerate(OPLOOPDIR):
            ds1=xr.open_dataset(path+exp+'/'+op+'/'+file1)
            ds1_list.append(ds1)      
        merged=xr.concat(ds1_list, dim='locations')
        # 
        opfilename=file1
        output_path=stackfolder +'/'+opfilename
        # Save the merged dataset to a new NetCDF file
        merged.to_netcdf(output_path)